In [1]:
import pandas
import re
import os
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import InputLayer, Masking, LSTM, TimeDistributed, Dense
from math import ceil
from keras.callbacks import Callback, ModelCheckpoint

Using TensorFlow backend.


In [2]:
csvs = list(filter(lambda file: re.match(r'^\d{8} Test Data\.txt$', file) is not None, os.listdir('./DynamometerData')))

In [3]:
dfs = [pandas.read_csv('./DynamometerData/' + csv, sep='\t', header=0) for csv in csvs]

In [5]:
max_length = (ceil(max([len(df) for df in dfs]) / 100)) * 100
max_length

61600

In [6]:
X = np.full([len(dfs), max_length, 3], -1.)
Y = np.full([len(dfs), max_length, 1], -1.)

for i, df in enumerate(dfs):

    # One sequence doesn't have this column as the brake is never touched
    if 'Brake_pressure_applied_PCM[]' in df.columns:
        X[i,:len(df),0] += df['Brake_pressure_applied_PCM[]'].values + 1
        
    X[i,:len(df),1] += df['Pedal_accel_pos_CAN[per]'].values + 1
    
    # The previous speed for the first measurement should always be 0
    X[i,0,2] += 1
    X[i,1:len(df),2] += df['Dyno_Spd[mph]'].values[:-1] + 1
    
    Y[i,:len(df),0] += df['Dyno_Spd[mph]'].values + 1

In [7]:
del dfs

In [9]:
X_std = np.zeros(X.shape)
Y_std = np.zeros(Y.shape)

X0_min, X0_max = X[:,:,0].min(), X[:,:,0].max()
X1_min, X1_max = X[:,:,1].min(), X[:,:,1].max()
X2_min, X2_max = X[:,:,2].min(), X[:,:,2].max()
Y0_min, Y0_max = Y[:,:,0].min(), Y[:,:,0].max()

X_std[:,:,0] += (X[:,:,0] - X0_min) / (X0_max - X0_min)
X_std[:,:,1] += (X[:,:,1] - X1_min) / (X1_max - X1_min)
X_std[:,:,2] += (X[:,:,2] - X2_min) / (X2_max - X2_min)
Y_std[:,:,0] += (Y[:,:,0] - Y0_min) / (Y0_max - Y0_min)

In [28]:
SPLIT = X_std.shape[0] - X_std.shape[0] // 10

indices = np.arange(0, X_std.shape[0])
np.random.shuffle(indices)

X_shuffled = X_std[indices,:,:]
Y_shuffled = Y_std[indices,:,:]

X_train, X_test = X_shuffled[:SPLIT,:,:], X_shuffled[SPLIT:,:,:]
Y_train, Y_test = Y_shuffled[:SPLIT,:,:], Y_shuffled[SPLIT:,:,:]

In [32]:
test_model = Sequential()

test_model.add(InputLayer(batch_input_shape=(1, 1, X.shape[2])))
test_model.add(LSTM(200, stateful=True))
test_model.add(Dense(Y.shape[2], activation='linear'))

test_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (1, 200)                  163200    
_________________________________________________________________
dense_3 (Dense)              (1, 1)                    201       
Total params: 163,401
Trainable params: 163,401
Non-trainable params: 0
_________________________________________________________________


In [71]:
test_model.load_weights('Models/MediumStateLSTM/E170L3.2593E-06.hdf5')

In [72]:
test_model.reset_states()

In [73]:
i = 1

Y_pred = np.zeros([1, Y_test.shape[1], Y_test.shape[2]])

inputs = X_train[i:i+1,0:1,:].copy()

j = 0
while X_train[i,j,0] != 0.:
    
    Y_pred[0,j:j+1,:] += test_model.predict(inputs, batch_size=1)

    inputs[0,0,:2] = X_train[i:i+1,j+1:j+2,:2]
    inputs[0,0,2] = Y_pred[0,j:j+1,0]
    
    j += 1
    
test_model.reset_states()

In [1]:
fig, ax = plt.subplots()

ax.scatter(range(max_length), Y_train[i,:,0], s=2)
ax.scatter(range(max_length), Y_pred[0,:,0], s=2)

# Zoom in
#plt.xlim(0, 10000)

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined